In [3]:
!pip install  --quiet kfp==1.8.22 tensorflow==2.3.0

In [17]:
! pip uninstall -y tensorflow==2.3.0  

Found existing installation: tensorflow 2.3.0
Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0


In [18]:
!pip install tensorflow==2.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.4/394.4 MB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 kB 41.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 33.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
# Restart the kernel to have the new kfp version installed

In [1]:
import kfp
from kfp import dsl
import kfp.components as components
# from kfp.components import InputPath, InputTextFile, OutputPath, OutputTextFile

from typing import NamedTuple

In [19]:
tf.__version__

'2.3.0'

In [20]:
import tensorflow as tf

type(tf.keras.saving.save_model)

AttributeError: module 'tensorflow.keras' has no attribute 'saving'

In [2]:
kfp.__version__

'1.8.22'

In [3]:

#-> NamedTuple("Output", [("MnistInputDirPath", str)])
# mnist_input_dir_path: OutputPath(str)
def put_public_mnist_to_local_minio() :
    # import numpy as np
    # import pandas as pd
    # import matplotlib.pyplot as plt
    # import tensorflow as tf
    # from tensorflow import keras

    import tempfile
    import os
    from minio import Minio
    import numpy as np
    import uuid

    # from minio.error import S3Error

    def get_minio_url():
        minio_host, minio_port = os.environ["MINIO_SERVICE_SERVICE_HOST"], os.environ["MINIO_SERVICE_SERVICE_PORT_HTTP"]
        minio_url= "{}:{}".format(minio_host, minio_port)
        return minio_url
    minio_url = get_minio_url()
    print("minio url:", minio_url)
    BUCKET_NAME="datapipeline-024"
    config = {"endpoint": minio_url,
        "access_key": "minio",
        "secret_key": "minio123",
        "secure": False}
    minio_client = Minio(**config)

    print("try to find bucket {}".format(BUCKET_NAME))
    found = minio_client.bucket_exists(BUCKET_NAME)
    print("found", found)
    if not found:
        minio_client.make_bucket(BUCKET_NAME)
    else:
        print("Bucket '{}' already exists".format(BUCKET_NAME))
    
    import tensorflow.keras as keras
    # import keras
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    #np.save generates file path with .npy suffix.
    temp_file_paths= ["/tmp/"+str(uuid.uuid4()) for i in range(4)]
    for file_path, data in zip(temp_file_paths,[x_train, y_train,x_test, y_test]):
        print("writing to ",file_path+".npy")
        np.save(file_path,data)

    
    
    minio_client.fput_object(BUCKET_NAME, "data/original/x_train.npy", temp_file_paths[0]+".npy")
    minio_client.fput_object(BUCKET_NAME, "data/original/y_train.npy", temp_file_paths[1]+".npy")
    minio_client.fput_object(BUCKET_NAME, "data/original/x_test.npy", temp_file_paths[2]+".npy")
    minio_client.fput_object(BUCKET_NAME, "data/original/y_test.npy", temp_file_paths[3]+".npy")
    
    for f in temp_file_paths:
        os.remove(f+".npy")

    # from collections import namedtuple
    # output = namedtuple('Output', ['MnistInputDirPath'])
    # return output(mnist_input_dir_path)
    

In [4]:

#-> NamedTuple("Output", [("MnistInputDirPath", str)])
# mnist_input_dir_path: OutputPath(str)
def reshape_mnist_and_put_back() :
    # import numpy as np
    # import pandas as pd
    # import matplotlib.pyplot as plt
    # import tensorflow as tf
    # from tensorflow import keras

    import tempfile
    import os
    from minio import Minio
    import numpy as np
    import uuid
    import glob
    
    def get_minio_url():
        minio_host, minio_port = os.environ["MINIO_SERVICE_SERVICE_HOST"], os.environ["MINIO_SERVICE_SERVICE_PORT_HTTP"]
        minio_url= "{}:{}".format(minio_host, minio_port)
        return minio_url
    minio_url = get_minio_url()
    print("minio url:", minio_url)
    
    BUCKET_NAME="datapipeline-024"
    config = {"endpoint": minio_url,
        "access_key": "minio",
        "secret_key": "minio123",
        "secure": False}
    minio_client = Minio(**config)

    
    random_prefix=str(uuid.uuid4())
    
    def download_path(filename):
        return "/tmp/{}_{}.npy".format(random_prefix,filename)
    
    for file_name in ["x_train", "x_test"]:
        minio_client.fget_object(BUCKET_NAME,  "data/original/{}.npy".format(file_name), download_path(file_name) )
        reshaped = np.load(download_path(file_name)).reshape(-1,28,28,1) / 255
        np.save(download_path(file_name+".reshaped"),reshaped)
        minio_client.fput_object(BUCKET_NAME, "data/original/{}.reshaped.npy".format(file_name), download_path(file_name+".reshaped"))

    

    # minio_client.fput_object(BUCKET_NAME,  "data/original/y_train.npy", download_path("y_train") )
    # minio_client.fput_object(BUCKET_NAME,  "data/original/y_test.npy", download_path("y_test") )
    # y_train = np.load(download_path("y_train")) / 255
    # y_test = np.load(download_path("y_test"))
    
   
    for f in glob.glob("/tmp/{}_*".format(random_prefix)):
        os.remove(f)

    # from collections import namedtuple
    # output = namedtuple('Output', ['MnistInputDirPath'])
    # return output(mnist_input_dir_path)
    

In [173]:
def train_model(epochs:int = 1, optimizer: str = "adam", model_save_prefix:str="models/trained/detect-digits", version:str = "1") :
    import tensorflow as tf
    import tensorflow.keras as keras
    import os
    from minio import Minio
    import numpy as np
    import uuid
    import glob
    import pandas as pd
    import json
    import shutil
# uncomment this function to have a better accuracy
#     def build_model():
#         model = keras.models.Sequential()
#         model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28,28,1)))
#         model.add(keras.layers.MaxPool2D(2, 2))

#         model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
#         model.add(keras.layers.MaxPool2D(2, 2))

#         model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
#         model.add(keras.layers.MaxPool2D(2, 2))

#         model.add(keras.layers.Flatten())
#         model.add(keras.layers.Dense(64, activation='relu'))

#         model.add(keras.layers.Dense(32, activation='relu'))

#         model.add(keras.layers.Dense(10, activation='softmax')) #output are 10 classes, numbers from 0-9
#         return model

    def build_model():
        model = keras.models.Sequential()
        model.add(keras.layers.Conv2D(10, (3, 3), activation='relu', input_shape=(28,28,1)))
        model.add(keras.layers.MaxPool2D(2, 2))

        model.add(keras.layers.Conv2D(10, (3, 3), activation='relu'))
        model.add(keras.layers.MaxPool2D(2, 2))

        model.add(keras.layers.Conv2D(10, (3, 3), activation='relu'))
        model.add(keras.layers.MaxPool2D(2, 2))

        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(10, activation='relu'))

        model.add(keras.layers.Dense(10, activation='relu'))

        model.add(keras.layers.Dense(10, activation='softmax')) #output are 10 classes, numbers from 0-9
        return model


    
    def compile_model(_model):
        _model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
        
    def upload_local_directory_to_minio(minio_client, local_path, bucket_name, minio_path):
        # assert os.path.isdir(local_path)

        for local_file in glob.glob(local_path + '/**'):
            local_file = local_file.replace(os.sep, "/") # Replace \ with / on Windows
            if not os.path.isfile(local_file):
                upload_local_directory_to_minio(
                    minio_client, local_file, bucket_name, minio_path + "/" + os.path.basename(local_file))
            else:
                remote_path = os.path.join(
                    minio_path, local_file[1 + len(local_path):])
                remote_path = remote_path.replace(
                    os.sep, "/")  # Replace \ with / on Windows
                minio_client.fput_object(bucket_name, remote_path, local_file)

    
    def get_minio_url():
        minio_host, minio_port = os.environ["MINIO_SERVICE_SERVICE_HOST"], os.environ["MINIO_SERVICE_SERVICE_PORT_HTTP"]
        minio_url= "{}:{}".format(minio_host, minio_port)
        return minio_url
    minio_url = get_minio_url()
    
    BUCKET_NAME="datapipeline-024"
    config = {"endpoint": minio_url,
        "access_key": "minio",
        "secret_key": "minio123",
        "secure": False}
    minio_client = Minio(**config)
    
    random_prefix=str(uuid.uuid4())
    def download_path(filename):
        return "/tmp/{}_{}.npy".format(random_prefix,filename)

    
    for file_name in ["y_train", "y_test"]:
        minio_client.fget_object(BUCKET_NAME,  "data/original/{}.npy".format(file_name), download_path(file_name) )
    y_train, y_test = np.load(download_path("y_train")), np.load(download_path("y_test"))
    
    for file_name in ["x_train", "x_test"]:
        minio_client.fget_object(BUCKET_NAME,  "data/original/{}.reshaped.npy".format(file_name), download_path(file_name) )
    x_train, x_test = np.load(download_path("x_train")), np.load(download_path("x_test"))
    
    
    model = build_model()
    compile_model(model)
    
    history = model.fit(
      x=x_train,
      y=y_train,
      epochs=epochs,
      batch_size=20,
    )
    
    model_saved_path="/tmp/saved_model_{}".format(random_prefix)
    # tf.saved_model.save(model, model_saved_path)
    # model.save(model, model_saved_path)
    tf.keras.models.save_model(model, model_saved_path)
    from os import listdir
    from os.path import isfile, join
    # onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    
    itisfile=isfile(model_saved_path)
    print("tf.__version__", tf.__version__)
    print("{} : it is file: {}".format( model_saved_path, itisfile) )
    if not itisfile:
        print ("it is directory")
        onlyfiles = [f for f in listdir(model_saved_path) ]
        print ("dir content: ")
        print(onlyfiles)
    
    if itisfile:
        print("saving as a file")
        minio_client.fput_object(BUCKET_NAME,"{}/{}/saved_model.pb".format(model_save_prefix,version), model_saved_path)
    else:
        print("saving as a directory")
        upload_local_directory_to_minio(minio_client, model_saved_path,BUCKET_NAME,"{}/{}".format(model_save_prefix,version)) 
    
    # tf.keras.saving.save_model(model, model_saved_path)
    
#     model.save(model_saved_path)
    # minio_client.fput_object(BUCKET_NAME, "models/trained/model.keras", model_saved_path)
    
    # print("zipping file:", model_saved_path+".zip")
    # print("zipping file:", model_saved_path+".zip")

    # shutil.make_archive(model_saved_path+".zip", 'zip', model_saved_path)
    # minio_client.fput_object(BUCKET_NAME, "models/trained/model.keras", model_saved_path)

    



In [169]:
from typing import NamedTuple

def test_model_and_save_metrics(model_save_prefix:str="models/trained/detect-digits", version:str="1") -> NamedTuple('Output', [('mlpipeline_ui_metadata', 'UI_metadata'),('mlpipeline_metrics', 'Metrics')]) :
    import tensorflow as tf
    import tensorflow.keras as keras
    import os
    from minio import Minio
    import numpy as np
    import uuid
    import glob
    import pandas as pd
    import json
    import shutil
    
    # generate confusion matrix csv
    def gen_cm_csv(y_test=None,test_predictions=None):
        confusion_matrix = tf.math.confusion_matrix(labels=y_test,predictions=test_predictions)
        confusion_matrix = confusion_matrix.numpy()
        vocab = list(np.unique(y_test))
        data = []
        for target_index, target_row in enumerate(confusion_matrix):
            for predicted_index, count in enumerate(target_row):
                data.append((vocab[target_index], vocab[predicted_index], count))

        df_cm = pd.DataFrame(data, columns=['target', 'predicted', 'count'])
        cm_csv = df_cm.to_csv(header=False, index=False)
        return cm_csv
    
    
    def get_minio_url():
        minio_host, minio_port = os.environ["MINIO_SERVICE_SERVICE_HOST"], os.environ["MINIO_SERVICE_SERVICE_PORT_HTTP"]
        minio_url= "{}:{}".format(minio_host, minio_port)
        return minio_url
    minio_url = get_minio_url()
    
    BUCKET_NAME="datapipeline-024"
    config = {"endpoint": minio_url,
        "access_key": "minio",
        "secret_key": "minio123",
        "secure": False}
    minio_client = Minio(**config)
    
    random_prefix=str(uuid.uuid4())
    def download_path(filename):
        return "/tmp/{}_{}.npy".format(random_prefix,filename)


    model_remote_path="{}/{}".format(model_save_prefix, version)
    model_saved_path="/tmp/{}/{}".format(model_save_prefix, version)
    # minio_client.fget_object(BUCKET_NAME,model_remote_path,model_saved_path)
    
    for bucket in minio_client.list_buckets():
        if bucket.name!=BUCKET_NAME:
            continue
        for item in minio_client.list_objects(bucket.name,model_remote_path,recursive=True):
            print("remote name:",item.object_name)
            print("local name:", "/tmp/"+item.object_name)
            minio_client.fget_object(bucket.name,item.object_name, "/tmp/"+item.object_name)
    
    
    from os import listdir
    from os.path import isfile, join

    

    itisfile=isfile(model_saved_path)
    print("{} : it is file: {}".format( model_saved_path, itisfile) )
    if not itisfile:
        print ("it is directory")
        onlyfiles = [f for f in listdir(model_saved_path) ]
        print ("dir content: ")
        print(onlyfiles)
    
    # model = tf.keras.models.load_model("/tmp/{}/{}".format(model_save_prefix, version))
    # model = tf.keras.saving.load_model("/tmp/{}/{}".format(model_save_prefix, version))
    print ("loading model in path:", model_saved_path)
    # model = tf.saved_model.load(model_saved_path)
    model = tf.keras.models.load_model(model_saved_path)
    
    for file_name in ["x_test", "y_test"]:
        minio_client.fget_object(BUCKET_NAME,  "data/original/{}.npy".format(file_name), download_path(file_name) )
    x_test, y_test = np.load(download_path("x_test")), np.load(download_path("y_test"))
    
    
    # Test the model against the test dataset
    # Returns the loss value & metrics values for the model in test mode.
    model_loss, model_accuracy = model.evaluate(x=x_test,y=y_test)
    
    
    # Generates output predictions for the input samples.
    test_predictions = model.predict(x=x_test)

    # Returns the indices of the maximum values along an axis.
    test_predictions = np.argmax(test_predictions,axis=1) 
    
    cm_csv = gen_cm_csv(y_test=y_test,test_predictions=test_predictions)
    
    #show model summary - how it looks
    stringlist = []
    model.summary(print_fn=lambda x: stringlist.append(x))
    metric_model_summary = "\n".join(stringlist)
    
    output_confussion_matrix = {
                "type": "confusion_matrix",
                "format": "csv",
                "schema": [
                    {'name': 'target', 'type': 'CATEGORY'},
                    {'name': 'predicted', 'type': 'CATEGORY'},
                    {'name': 'count', 'type': 'NUMBER'},
                  ],
                "target_col" : "actual",
                "predicted_col" : "predicted",
                "source": cm_csv,
                "storage": "inline",
                "labels": list(np.arange(10)) #0..9 labels
            }
    output_model_summary = {
                'type': 'markdown',
                'storage': 'inline',
                'source': f'''# Model Overview
## Model Summary

```
{metric_model_summary}
```

## Model Performance

**Accuracy**: {model_accuracy}
**Loss**: {model_loss}

'''
            }
    
    metadata = {"outputs": [output_confussion_matrix, output_model_summary]}
    metrics = {
      'metrics': [{
          'name': 'model_accuracy',
          'numberValue':  float(model_accuracy),
          'format' : "PERCENTAGE"
        },{
          'name': 'model_loss',
          'numberValue':  float(model_loss),
          'format' : "PERCENTAGE"
        }]}
    
    
    class NpJsonEncoder(json.JSONEncoder):
        """Serializes numpy objects as json."""

        def default(self, obj):
            if isinstance(obj, np.integer):
                return int(obj)
            elif isinstance(obj, np.bool_):
                return bool(obj)
            elif isinstance(obj, np.floating):
                if np.isnan(obj):
                    return None  # Serialized as JSON null.
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            else:
                return super().default(obj)
        
    from collections import namedtuple
    output = namedtuple('Output', ['mlpipeline_ui_metadata', 'mlpipeline_metrics'])
    return output(json.dumps(metadata, cls=NpJsonEncoder),json.dumps(metrics, cls=NpJsonEncoder))

In [170]:

import kfp
from kfp import dsl
import kfp.components as components

step_put_public_mnist_to_local_minio = components.create_component_from_func(put_public_mnist_to_local_minio,base_image="kubeflownotebookswg/jupyter-tensorflow-full")
step_reshape_mnist_and_put_back = components.create_component_from_func(reshape_mnist_and_put_back,base_image="kubeflownotebookswg/jupyter-tensorflow-full")
step_train_model = components.create_component_from_func(train_model,base_image="kubeflownotebookswg/jupyter-tensorflow-full")
step_test_model_and_save_metrics = components.create_component_from_func(test_model_and_save_metrics,base_image="kubeflownotebookswg/jupyter-tensorflow-full")

@dsl.pipeline(
    name='kfp-pipeline-digits-clsfier',
    description='Classify digits'
)
def kfp_pipeline_start(epochs,optimizer,model_save_prefix, version):
    step1 = step_put_public_mnist_to_local_minio()
    step2 = step_reshape_mnist_and_put_back()
    step3 = step_train_model(epochs,optimizer,model_save_prefix, version)
    step4 = step_test_model_and_save_metrics(model_save_prefix, version)

    step2.after(step1)
    step3.after(step2)
    step4.after(step3)






In [171]:
import os
host, port = os.environ["ML_PIPELINE_PORT_8888_TCP_ADDR"],os.environ["ML_PIPELINE_SERVICE_PORT_HTTP"]
kfp_endpoint="http://{}:{}".format(host,port)

kfp_client = kfp.Client(host=kfp_endpoint)
pipeline_arguments = {
    "epochs": 1,
    "optimizer": "adam",
    "model_save_prefix": "models/trained/detect-digits",
    "version": "13"
}
pipeline_run = kfp_client.create_run_from_pipeline_func(kfp_pipeline_start, arguments=pipeline_arguments)


In [172]:
import os
from IPython.display import IFrame
cluster_id= os.environ["MEDDASH_CLUSTER_ID"]
run_id=pipeline_run.run_id
print("pipeline.run_id:", run_id)
pipeline_vis_endpoint="http://kf-ml-pipeline-ui.{}.hosted.meddash.cloud/#/runs/details/{}".format(cluster_id,run_id)
IFrame(pipeline_vis_endpoint, width=1200, height=600)

pipeline.run_id: b609cbea-7d86-49ef-af49-44a09c6de283


In [105]:
import tensorflow as tf
import tensorflow.keras as keras
import os
from minio import Minio
import numpy as np
import uuid
import glob
import pandas as pd
import json
import shutil

def get_minio_url():
    minio_host, minio_port = os.environ["MINIO_SERVICE_SERVICE_HOST"], os.environ["MINIO_SERVICE_SERVICE_PORT_HTTP"]
    minio_url= "{}:{}".format(minio_host, minio_port)
    return minio_url
minio_url = get_minio_url()

BUCKET_NAME="datapipeline-024"
config = {"endpoint": minio_url,
    "access_key": "minio",
    "secret_key": "minio123",
    "secure": False}
minio_client = Minio(**config)

random_prefix=str(uuid.uuid4())
def download_path(filename):
    return "/tmp/{}_{}".format(random_prefix,filename)

    

version="5"

for bucket in minio_client.list_buckets():
    if bucket.name!=BUCKET_NAME:
        continue
    for item in minio_client.list_objects(bucket.name,"models/trained/detect-digits/5",recursive=True):
        print(item.object_name)
        minio_client.fget_object(bucket.name,item.object_name,"/tmp/"+item.object_name)


# minio_client.fget_object(BUCKET_NAME, "models/trained/detect-digits/5", download_path("model.keras"))


bucket name: datapipeline-024
models/trained/detect-digits/5/saved_model.pb
models/trained/detect-digits/5/variables/variables.data-00000-of-00001
models/trained/detect-digits/5/variables/variables.index
bucket name: mlpipeline


In [106]:
!ls /tmp/models/trained/detect-digits/

5


In [104]:
!rm -rf /tmp/models